In [1]:
import torch
import numpy as np

from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

In [2]:
LR = 1e-6
MAX_EPOCH = 50
BATCH_SIZE = 512

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
class SineApproximator(nn.Module):
    def __init__(self):
        super(SineApproximator, self).__init__()
        self.regressor = nn.Sequential(nn.Linear(1, 1024),
                                       nn.ReLU(inplace=True),
                                       nn.Linear(1024, 1024),
                                       nn.ReLU(inplace=True),
                                       nn.Linear(1024, 1))
    def forward(self, x):
        output = self.regressor(x)
        return output


In [5]:
X = np.random.rand(4*10**5)*2*np.pi
y = np.sin(X)

X_train, X_val, y_train, y_val = map(torch.tensor, train_test_split(X, y, test_size=0.2))

train_dataloader = DataLoader(TensorDataset(X_train.unsqueeze(1), 
                                            y_train.unsqueeze(1)), 
                              batch_size=BATCH_SIZE,
                              pin_memory=True, 
                              shuffle=True)
val_dataloader = DataLoader(TensorDataset(X_val.unsqueeze(1), 
                                          y_val.unsqueeze(1)), 
                            batch_size=BATCH_SIZE,
                            pin_memory=True, 
                            shuffle=True)

In [6]:
model = SineApproximator().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss(reduction="mean")

In [ ]:
# training loop
train_loss_list = list()
val_loss_list = list()
for epoch in range(MAX_EPOCH):
    print("epoch %d / %d" % (epoch+1, MAX_EPOCH))
    model.train()
    # training loop
    temp_loss_list = []
    for X_train, y_train in train_dataloader:
        X_train = X_train.type(torch.float32).to(device)
        y_train = y_train.type(torch.float32).to(device)

        optimizer.zero_grad()

        score = model(X_train)
        loss = criterion(input=score, target=y_train)
        loss.backward()

        optimizer.step()

        temp_loss_list.append(loss.detach().cpu().numpy())
    print(temp_loss_list[-1])

epoch 1 / 50
0.15118885
epoch 2 / 50
0.10762521
epoch 3 / 50
0.09278396
epoch 4 / 50
0.08650895
epoch 5 / 50
0.07574762
epoch 6 / 50
0.060834385
epoch 7 / 50
0.050784707
epoch 8 / 50


In [ ]:
p = torch.linspace(0,2*np.pi, 40)
p = p.unsqueeze(1)

In [ ]:
px = model(p).detach().numpy()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(p,px)